# Verilog Parsar

## Introduction
- This is a verilog parser written in python.
- It can parse verilog file and generate a report

## Usage
- Check for the following:
    - Unreachable Blocks
    - Uninitialized Register
    - Inferring Latches
    - Unreachable State
    - Non Full Case
    - Non Parallel Case
    - Multiple Drivers
    - Arithmetic Overflow
    - Integer Overflow

# Imports

In [121]:
import re
import sys
import os

In [122]:
def file_reader(file_name):
    x = []
    try:
        # Open the file in read mode
        with open(file_name, 'r') as file:
            # Read and print each line
            for line in file:
                x.append(line.strip())  

    except FileNotFoundError:
        print(f"File not found: {file_name}")

    except Exception as e:
        print(f"An error occurred: {e}")
    
    return x


### Testing reading verilog file

In [123]:
file_path = 'tst.v'

verilog_code = file_reader(file_path)
    
for i in verilog_code:
        print(i)

module UnreachableBlocks();
reg reach;
wire state;
wire data;

initial
begin
reach = 1'b1;
end

always @(state)
begin
if (reach == 2'b0)
begin
data <= 1'b1;
end
end
endmodule

module UninitializedRegister();
wire data;

initial
begin
$display("%b", data);
end
endmodule

module InferringLatches();
reg out;
wire enable, Data;

always @(enable)
begin
if (enable)
begin
out <= Data;
end
end
endmodule

module UnreachableState(input clk);
reg [1:0] current_state, next_state;
localparam [1:0] S1 = 2'b00 ;
localparam [1:0] S2 = 2'b01 ;
localparam [1:0] S3 = 2'b10 ;

always @(posedge clk)
begin
current_state <= next_state;
end

always @(*)
begin
case (current_state)
S1:
begin
next_state <= S2;
end
S2:
begin
next_state <= S1;
end
S3
begin
next_state <= S1;
end

endcase
end
endmodule

module NonFullCase();
reg [1:0] x, y;

always @(*)
begin
case(x)
2'b00: y = 1'b00;
2'b01: y = 1'b01;
// Missing cases for '10' & '11'
endcase
end
endmodule

module NonParallelCase();
reg [1:0] x, y;

always @(*)
begi

### Dividing Modules

#### Utility Functions

In [124]:
def counting_modules(verilog_code):
    module_counter = 0
    for i in verilog_code:
        if i == "endmodule":
            module_counter += 1
    return module_counter

In [125]:
def creating_module_lists(verilog_code, module_lists, module_counter):
    for i in verilog_code:
        module_lists[len(module_lists)-module_counter].append(i)
        if i == "endmodule":
            module_counter -= 1
    return module_lists

In [126]:
def remove_empty_strings(module_lists):
    for i in module_lists:
     while("" in i) : 
        i.remove("")
    return module_lists

In [127]:
def remove_comments(module_lists):
    i_counter = 0
    for i in module_lists:
        
        j_counter = 0
        
        for j in i:
            
            if j.startswith('//'):
                i.remove(j)
            
            elif '//' in j:
                index = j.find('//')
                j = j[:index]
                module_lists[i_counter][j_counter] = j
            
            j_counter += 1

        i_counter += 1
    return module_lists

#### Applying utility functions

In [128]:
module_counter = counting_modules(verilog_code)

In [129]:
# create lists according to module counter
module_lists = [[] for i in range(module_counter)]
module_lists

[[], [], [], [], [], [], [], [], []]

In [130]:
# append each module to a module_lists from the y
module_lists = creating_module_lists(verilog_code, module_lists, module_counter)

In [131]:
# remove all empty strings from the module_lists
module_lists = remove_empty_strings(module_lists)

In [132]:
module_lists = remove_comments(module_lists)

In [133]:
module_lists

[['module UnreachableBlocks();',
  'reg reach;',
  'wire state;',
  'wire data;',
  'initial',
  'begin',
  "reach = 1'b1;",
  'end',
  'always @(state)',
  'begin',
  "if (reach == 2'b0)",
  'begin',
  "data <= 1'b1;",
  'end',
  'end',
  'endmodule'],
 ['module UninitializedRegister();',
  'wire data;',
  'initial',
  'begin',
  '$display("%b", data);',
  'end',
  'endmodule'],
 ['module InferringLatches();',
  'reg out;',
  'wire enable, Data;',
  'always @(enable)',
  'begin',
  'if (enable)',
  'begin',
  'out <= Data;',
  'end',
  'end',
  'endmodule'],
 ['module UnreachableState(input clk);',
  'reg [1:0] current_state, next_state;',
  "localparam [1:0] S1 = 2'b00 ;",
  "localparam [1:0] S2 = 2'b01 ;",
  "localparam [1:0] S3 = 2'b10 ;",
  'always @(posedge clk)',
  'begin',
  'current_state <= next_state;',
  'end',
  'always @(*)',
  'begin',
  'case (current_state)',
  'S1:',
  'begin',
  'next_state <= S2;',
  'end',
  'S2:',
  'begin',
  'next_state <= S1;',
  'end',
  'S3',

### Unreachable State

### Non-Full Case

### Unintialized registers

### Inferring Latches

### Unreachable Blocks

### Multiple Drivers

### Arithmetic Overflow

### Non-Parallel Case